In [1]:
import ee
try:
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
except:
    ee.Authenticate()
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

from e_sbae.sampling import grid
from e_sbae.ee import time_series

/home/sepal-user/venv/eSBAE/lib/python3.8/site-packages/bfast/__init__.py:10: UserWarning: cupy is not available in this environment, GPU fonctionnalities won't be available
  warn("cupy is not available in this environment, GPU fonctionnalities won't be available")


In [2]:
#aoi = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq('ADM1_NAME', 'Beni'))
#cells, points = grid.hexagonal_grid(aoi, 8)
#grid.plot_samples(aoi, points, cells)
import geopandas as gpd
#points = gpd.read_file("/home/sepal-user/Ethiopia/grids/fine_grid_points_biomes_res15_esri54017.gpkg")
#points.insert(0, 'point_id', range(0, 0 + len(points)))
#points.drop(['index_right', 'Class_code', 'name'], axis=1, inplace=True)
#len(points)

In [3]:
aoi = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq('ADM1_NAME', 'Oromia'))
points = ee.FeatureCollection(
    "users/andreasvollrath/eth_ts_analysis/ethiopia_hex_centroids_dggrid_15"
).filterBounds(aoi).sort('point_id')
#points = ee.FeatureCollection("users/andreasvollrath/01_test_sbae_points").sort('point_id')

In [7]:
points.size().getInfo()

91209

In [4]:
# start of calibration period (mainly for bfast)
start_calibration = "2013-01-01"  # YYYY-MM-DD format

# Actual period of interest, i.e. monitoring period
start_monitor =     "2018-01-01"  # YYYY-MM-DD format
end_monitor =       "2023-03-01"  # YYYY-MM-DD format

# Directory where output and temp files will go
outdir = 'results'  # goes to module_results/sbae_point_analysis if set to None

# Select algorithms to run (True or False)
cusum_deforest =  True
bfast_monitor =   True
bs_slope =        True
ts_metrics =      True
ccdc =            True
landtrendr =      True
jrc_nrt =         True
global_products = True

# select the bands to extract
bands = ['green', 'red', 'nir', 'swir1', 'swir2', 'ndfi', 'brightness', 'greenness', 'wetness'] # other choices: ndfi, ndmi, mndwi, brightness, greenness, wetness
#bands = ['nir', 'ndfi'] # other choices: ndfi, ndmi, mndwi, brightness, greenness, wetness

# select the band for univariate ts-analysis (has to be inside bands list)
ts_band = 'ndfi'

# select the resolution to which the satellite data will be resized.
scale = 30  # in meters (70 m is half ha, relates to FAO forest definition)

### DO NOT CHANGE YET ###
satellite='Landsat'  # this is going to be Surface Reflactance, Collection 2, Tier 1 data only
max_cloud_cover = 75  # in percentage (0-100)

point_id_name = 'point_id'

In [5]:
# landsat related parameters
lsat_params = {
    'l9':     True,
    'l8':     True,
    'l7':     True,
    'l5':     True,
    'l4':     True,
    'brdf':   True,
    'bands':  bands,
    'max_cc': max_cloud_cover
}

# bfast parameters
bfast_params = {
    'run':           bfast_monitor,
    'start_monitor': start_monitor, 
    'freq':          365,
    'k':             3, 
    'hfrac':         0.25, 
    'trend':         True, 
    'level':         0.05, 
    'backend':       'python'
}

# cusum parameters
cusum_params = {
    'run':              cusum_deforest,
    'nr_of_bootstraps': 1000
}

# slope parameters
bs_slope_params = {
    'run':              bs_slope,
    'nr_of_bootstraps': 1000
}

# time-series metrics
ts_metrics_params = {
    'run':              ts_metrics,
    'outlier_removal':  False,
    'z_threshhold':     3
}

# ccdc parameters
ccdc_params = {
    'run':                   ccdc,
    'breakpointBands':       ['green', 'red', 'nir', 'swir1', 'swir2'],
    'tmaskBands':            ['green', 'swir2'],
    'minObservations':       6,
    'chiSquareProbability':  .99,
    'minNumOfYearsScaler':   1,
    'dateFormat':            2,
    'lambda':                20,
    'maxIterations':         1000
}


landtrendr_params = { 
        'run':                    landtrendr,
        'maxSegments':            6,
        'spikeThreshold':         0.9,
        'vertexCountOvershoot':   3,
        'preventOneYearRecovery': True,
        'recoveryThreshold':      0.25,
        'pvalThreshold':          0.05,
        'bestModelProportion':    0.75,
        'minObservationsNeeded':  3
}

jrc_nrt_params = {
    'run': jrc_nrt
}

# global products parameters
global_products = {
    'run':                      global_products,
    'gfc':                      True,     # will include tree-cover 2000, loss, gain, lossyear
    'tmf':                      True,    # will include deforestation and degradation year for tropical moist forests
    'tmf_years':                True,    # will include classes per year - according to the monitor period
    'esa_lc20':                 True,    # will include ESA LandCover Product class
    'copernicus_lc':            True,    # will include ESA LandCover Product class - acording to the monitoring years
    'esri_lc':                  True,    # will include the classes from ESRI World Cover 2020
    'lang_tree_height':         True,    # returns the Tree Height from Lang et al 2022
    'potapov_tree_height':      True,    # returns the tree height from Potapov et al. 2019 
    'elevation':                True,    # returns elevation, slope and aspect
    'dynamic_world_tree_prob':  True,    # returns Min, Max, Mean and StdDev of the trees probability for the monitoring period
    'dynamic_world_class_mode': True     # returns the mode of the class for the monitoring period   
}

### DO NOT CHANGE ###
### GATHER ALL INFO INTO A DICT #####
config_dict = {
    'work_dir':                         outdir,
    'workers':                          10,
    'max_points_per_chunk':             300,
    'grid_size_levels':                 [0.25, 0.125, 0.075],  # definition of chunk sizes in degrees  
    'lsat_params':                      lsat_params,
    'ts_params': {
        'start_calibration':            start_calibration,
        'start_monitor':                start_monitor,
        'end_monitor':                  end_monitor,
        'point_id':                     point_id_name,
        'bands':                        bands,
        'ts_band':                      ts_band,
        'satellite':                    satellite,
        'scale':                        scale,
        'max_cc':                       max_cloud_cover,
        'outlier_removal':              True,
        'smooth_ts':                    True       
    },    
    'bfast_params':                     bfast_params,
    'cusum_params':                     cusum_params,
    'bs_slope_params':                  bs_slope_params,
    'ts_metrics_params':                ts_metrics_params,
    'ccdc_params':                      ccdc_params,
    'landtrendr_params':                landtrendr_params,
    'jrc_nrt_params':                   jrc_nrt_params,
    'global_products':                  global_products
}

In [ ]:
from pathlib import Path
import json
outdir = Path(outdir)
outdir.mkdir(parents=True, exist_ok=True)

# create tmpdir and outdir
#tmpdir = outdir.joinpath("tmp")
#tmpdir.mkdir(parents=True, exist_ok=True)

# create config file
config_file = outdir.joinpath("config.json")
with open(config_file, "w") as f:
    json.dump(config_dict, f)

time_series.cascaded_extraction_ee(points, config_file)

In [8]:
from e_sbae.helpers import ee as eeh
# get users asset root
asset_root = ee.data.getAssetRoots()[0]["id"]
for a in ee.data.listAssets({"parent": f"{asset_root}"})["assets"]:
    #print(a["name"].split('/')[-1][:9])
    if a["name"].split('/')[-1][:9] == "tmp_esbae":
        print(a["name"])
        eeh.delete_sub_folder(a["name"].split('/')[-1])

projects/earthengine-legacy/assets/users/andreasvollrath/tmp_esbae_20740449
INFO: Removing assets within asset folder projects/earthengine-legacy/assets/users/andreasvollrath/tmp_esbae_20740449
INFO: Removing asset folder projects/earthengine-legacy/assets/users/andreasvollrath/tmp_esbae_20740449.
projects/earthengine-legacy/assets/users/andreasvollrath/tmp_esbae_5c322494
INFO: Removing assets within asset folder projects/earthengine-legacy/assets/users/andreasvollrath/tmp_esbae_5c322494
INFO: Removing asset folder projects/earthengine-legacy/assets/users/andreasvollrath/tmp_esbae_5c322494.
projects/earthengine-legacy/assets/users/andreasvollrath/tmp_esbae_748564d4
INFO: Removing assets within asset folder projects/earthengine-legacy/assets/users/andreasvollrath/tmp_esbae_748564d4
INFO: Removing asset folder projects/earthengine-legacy/assets/users/andreasvollrath/tmp_esbae_748564d4.
projects/earthengine-legacy/assets/users/andreasvollrath/tmp_esbae_87a26251
INFO: Removing assets withi

In [ ]:
from pathlib import Path
import json
import geojson
import geopandas as gpd
import pandas as pd
files = []
for file in Path(outdir).joinpath('tmp').glob('*geojson'):
    with open(file, 'r') as outfile:
        d = gpd.GeoDataFrame.from_features(geojson.loads(geojson.load(outfile))).drop(['dates', 'ts', 'images', 'geometry'], axis=1)
        files.append(d)
    
newdf = pd.concat(files)

In [ ]:
points[~points.point_id.isin(newdf.point_id.to_list())]

In [ ]:
newdf.drop_duplicates('point_id').sort_values('point_id')